In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

Step 1: Define the Lattice Structure
- Lattice Shape (number indicates a dipole exists, 0 indicates no dipole)
- Spin states (each dipole will be represented by an entry in the lattice with a value of either +1 or -1)

Step 2: Hamiltonian Construction

- The hamiltonian describes the total energy of the system, accounting for interactions between neightboring spins. The enerrgy interaction between two neightboring spins is typically proportional to the product of their spin states (i.e. $H = -J\sum S_iS_j$, where $J$ is a coupling constant and $S_i$ and $S_j$ are the spin states of neighbors.)
- Once lattice is constructed, we need to filter out which dipoles are neighbors. For a 2D array, a dipole can interact with its left, right, top, below, top right, top left, bottom right, bottom left neighbors.

Step 3: Calculate total energy

- Use $H = -J\sum S_iS_j$ formula and sum up the energy contributions from every dipole in the lattice to determine the "total energy"

In [2]:
def print_lattice(lattice):
    max_cols = max(len(row) for row in lattice)
    
    # Calculate spacing for alignment
    for i, row in enumerate(lattice):
        # Print leading spaces for proper alignment based on row index
        print(' ' * (max_cols - len(row)), end='')  
        for val in row:
            if val == 0:
                print('  ', end=' ')  # Empty space for zero dipoles
            else:
                print(f'{val:2}', end=' ')  # Format positive and negative values
        print()  # Move to the next line

#Step 1: Define the Lattice Structure.
#1 = up spin, -1 = down spin, 0 = no dipole

lattice = np.array([
    [1, 1, 1],  # Row 0: 3 dipole (top)
    [1, 1, 1],  # Row 1: 3 dipole (middle)
    [1, 1, 1],  # Row 2: 3 dipoles (bottom)
])

lattice_2 = np.array([
    [-1, -1, -1],  # Row 0: 3 dipole (top)
    [-1, -1, -1],  # Row 1: 3 dipole (middle)
    [-1, -1, -1],  # Row 2: 3 dipoles (bottom)
])

#Step 2: Hamiltonian Construction

def hamiltonian(lattice, J):
    
    total_energy = 0
    rows = len(lattice)
    cols = len(lattice[0])

    for i in range(rows):
        for j in range(cols):
            if lattice[i][j] != 0:  # Only consider if current index is a dipole
                
                # Check neighbors (up, down, left, right, and diagonals)
                neighbors = [
                    (i - 1, j),   # Up
                    (i + 1, j),   # Down
                    (i, j - 1),   # Left
                    (i, j + 1),   # Right

                    (i - 1, j - 1), # Up-Left
                    (i - 1, j + 1), # Up-Right
                    (i + 1, j - 1), # Down-Left
                    (i + 1, j + 1)  # Down-Right
                ]
                
                for ni, nj in neighbors:
                    if 0 <= ni < rows and 0 <= nj < cols:  # Check boundaries
                        if lattice[ni][nj] != 0:  # Check if neighbor has a dipole

                            #Calculate energy contribution based on spin alignment
                            total_energy += -J * lattice[i][j] * lattice[ni][nj]

    return total_energy*1/2

# Step 3: Calculate total energy

J = 1  #Coupling constant
total_energy = hamiltonian(lattice, J)
total_energy_2 = hamiltonian(lattice_2, J)

print_lattice(lattice)
print("\nTotal Energy:", total_energy)

print('\n')
print_lattice(lattice_2)
print("\nTotal Energy:", total_energy_2)

 1  1  1 
 1  1  1 
 1  1  1 

Total Energy: -20.0


-1 -1 -1 
-1 -1 -1 
-1 -1 -1 

Total Energy: -20.0


In [3]:
lattice = np.array([
    [0, 0, 1, 1, 1],  # Row 0: 3 dipole (top)
    [0, 1, 1, 1, 0],  # Row 1: 3 dipole (middle)
    [1, 1, 1, 0, 0],  # Row 2: 3 dipoles (bottom)
])

J = 1  #Coupling constant
total_energy = hamiltonian(lattice, J)

print_lattice(lattice)
print("\nTotal Energy:", total_energy)

       1  1  1 
    1  1  1    
 1  1  1       

Total Energy: -18.0


In [4]:
def generate_dipole_combinations(lattice):
    '''chatGPT wrote this function because I was lazy, but i will rewrite my own if bad idea.'''

    # Find the indices of dipoles (non-zero entries)
    indices = np.argwhere(lattice != 0)

    # Initialize a list to hold all possible lattices
    all_lattices = []

    # Recursive helper function
    def recursive_fill(current_lattice, index):
        # If we have filled all dipole positions, add the current lattice to the list
        if index == len(indices):
            all_lattices.append(np.copy(current_lattice))
            return
        
        # Get the current dipole's position
        pos = indices[index]
        
        # Set the current dipole to -1 and call recursively
        current_lattice[tuple(pos)] = -1
        recursive_fill(current_lattice, index + 1)
        
        # Set the current dipole to +1 and call recursively
        current_lattice[tuple(pos)] = 1
        recursive_fill(current_lattice, index + 1)
        
        # Reset to zero for consistency (optional)
        current_lattice[tuple(pos)] = 0

    # Start the recursive filling with the original lattice
    recursive_fill(lattice, 0)

    return all_lattices


all_lattices = generate_dipole_combinations(lattice)
print("The total amount of lattices possible is %i, which makes sense because we have 9 dipoles and 2^9 = 512." %len(all_lattices))

J = 1  #Coupling constant

#loops through each lattice
for i, lat in enumerate(all_lattices):
    energy = hamiltonian(all_lattices[i], J)
    print('\nLattice arrangement: %i\n'%(i+1))
    print_lattice(all_lattices[i])
    print("\nTotal Energy: %i"%energy)

The total amount of lattices possible is 512, which makes sense because we have 9 dipoles and 2^9 = 512.

Lattice arrangement: 1

      -1 -1 -1 
   -1 -1 -1    
-1 -1 -1       

Total Energy: -18

Lattice arrangement: 2

      -1 -1 -1 
   -1 -1 -1    
-1 -1  1       

Total Energy: -10

Lattice arrangement: 3

      -1 -1 -1 
   -1 -1 -1    
-1  1 -1       

Total Energy: -10

Lattice arrangement: 4

      -1 -1 -1 
   -1 -1 -1    
-1  1  1       

Total Energy: -6

Lattice arrangement: 5

      -1 -1 -1 
   -1 -1 -1    
 1 -1 -1       

Total Energy: -14

Lattice arrangement: 6

      -1 -1 -1 
   -1 -1 -1    
 1 -1  1       

Total Energy: -6

Lattice arrangement: 7

      -1 -1 -1 
   -1 -1 -1    
 1  1 -1       

Total Energy: -10

Lattice arrangement: 8

      -1 -1 -1 
   -1 -1 -1    
 1  1  1       

Total Energy: -6

Lattice arrangement: 9

      -1 -1 -1 
   -1 -1  1    
-1 -1 -1       

Total Energy: -8

Lattice arrangement: 10

      -1 -1 -1 
   -1 -1  1    
-1 -1  1     